In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

In [ ]:
df_legend = pd.read_excel('reports/supp_tables.xlsx', sheet_name=6)
df_legend

In [ ]:
df = list()

for i in snakemake.input['pred']:
    _df = pd.read_csv(i)
    stats = spearmanr(_df['Predicted Melting Point'], _df['True Melting Point'])
    method = i.split('=')[1].split('_')[0].replace('1ve1', '1v')
    df.append({
        'method': method,
        'Correlation': stats.correlation,
        'lora': 'Head Only'
    })

for i in snakemake.input['pred_lora']:
    _df = pd.read_csv(i)
    stats = spearmanr(_df['Predicted Melting Point'], _df['True Melting Point'])
    method = i.split('=')[1].split('_')[0].replace('1ve1', '1v')
    df.append({
        'method': method,
        'Correlation': stats.correlation,
        'lora': 'Head + LoRA'
    })

df = pd.DataFrame(df)
df = pd.concat([df_legend, df]).reset_index(drop=True)
df.to_csv(snakemake.output['stats'], index=False)

In [ ]:
plt.figure(figsize=(4, 4), dpi=300)
palette = ["#87c785", "#572f30"]
plot = sns.barplot(data=df, x='method', y='Correlation', hue='lora', palette=palette)
plt.xlabel('Model')
sns.despine()
plt.legend(title=None, loc='lower right')
plt.savefig(snakemake.output['fig'], bbox_inches='tight', dpi=300, transparent=True)